In [53]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer 

### Loading data

In [54]:
data = pd.read_csv('./nlp-getting-started/test.csv', encoding='utf-8', error_bad_lines=False)
# pd.set_option('display.max_rows', None)
data.describe(include='all')

b'Skipping line 886: expected 4 fields, saw 7\n'


,id,keyword,location,text
count,3264.000000,3238,2160,3264
unique,NaN,221,1600,3243
top,NaN,deluged,New York,11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charge...
freq,NaN,23,38,3
mean,5434.344363,NaN,NaN,NaN
std,3173.281076,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN
25%,2680.250000,NaN,NaN,NaN
50%,5502.000000,NaN,NaN,NaN
75%,8178.250000,NaN,NaN,NaN


In [55]:
data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, stay safe everyone."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


### Data Cleaning

In [56]:
#punctutation removal
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# Tockenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

# stopword removal 
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text


# return to string
def listToString(s):     
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

# Lematizing the words
lemmatizer = WordNetLemmatizer()

def lementization(text):
    text = lemmatizer.lemmatize(text)
    return text


In [57]:
def DataCleaningFunction(text):
    text = clean_text(text)
    text = tokenization(text.lower())
    text = remove_stopwords(text)
    text = listToString(text)
    text = lementization(text.lower())
    return text


In [58]:
df = data.copy()
df['text'] = df['text'].apply(lambda x: DataCleaningFunction(x))
df.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different cities stay safe everyone
2,3,NaN,NaN,forest fire spot pond geese fleeing across street cannot save
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills china taiwan


In [59]:
test_x = df['text']
test_data_uncleaned = df['text']

In [60]:
test_x = np.array(test_x)
test_data_uncleaned = np.array(test_data_uncleaned)
test_x

array(['happened terrible car crash',
       'heard earthquake different cities stay safe everyone',
       'forest fire spot pond geese fleeing across street cannot save',
       ..., 'green line derailment chicago ',
       'meg issues hazardous weather outlook hwo ',
       'cityofcalgary activated municipal emergency plan yycstorm'],
      dtype=object)

### loading dictionary and model

In [61]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=5000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/glove_models_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [62]:
json_file = open('./NN_Models/Glove_BLSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/Glove_BLSTM_model.h5')


#### useful functions

In [63]:
def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1

In [64]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])

    return wordIndices

## classification

In [66]:

f = open("output.txt","w")
for index_of_interest, text1 in enumerate(test_x):
    tweetText = text1

    tweetText = convert_text_to_index_array(tweetText)

    input = pad_sequences([tweetText], padding='post', maxlen=100)
    
    pred = model.predict_generator(input)

    print(test_data_uncleaned[index_of_interest]+' :'+ labels[rounding(pred[0])]+' Tweet \n' )
    f.write(test_data_uncleaned[index_of_interest]+' :'+ labels[rounding(pred[0])]+' Tweet \n' )

happened terrible car crash :real Tweet 

heard earthquake different cities stay safe everyone :real Tweet 

forest fire spot pond geese fleeing across street cannot save :real Tweet 

apocalypse lighting spokane wildfires :real Tweet 

typhoon soudelor kills china taiwan :real Tweet 

shakingits earthquake :real Tweet 

theyd probably still show life arsenal yesterday eh eh :fake Tweet 

hey :fake Tweet 

nice hat :fake Tweet 

fuck :fake Tweet 

dont like cold :fake Tweet 

nooooooooo dont :fake Tweet 

dont tell :fake Tweet 

 :fake Tweet 

awesome :fake Tweet 

birmingham wholesale market ablaze bbc news fire breaks birminghams wholesale market  :fake Tweet 

sunkxssedharry wear shorts race ablaze  :fake Tweet 

previouslyondoyintv toke makinwa ûªs marriage crisis sets nigerian twitter ablaze  :fake Tweet 

check nsfw :fake Tweet 

psa ûªm splitting personalities techies follow ablazeco burners follow ablaze :fake Tweet 

beware world ablaze sierra leone amp guap :fake Tweet 

burn

jackienatalydlt get iced annihilation :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

incindia sonia gandhi like pakistan every defeat treated like victorytotal annihilation answer  :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

stop annihilation salt river wild horses via change :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via change :fake Tweet 

us national park services tonto national forest stop annihilation salt river wild horse via ch

christian attacked muslims temple mount waving israeli flag via pamela geller  :real Tweet 

attacked robotlvl ive earned total free satoshis robotcoingame bitcoin freebitcoin :fake Tweet 

attitude problem result constantly belittled bashed attacked amp demoralized fuck  :fake Tweet 

mustve forgot memo getting attacked resident receiving concussion funny :fake Tweet 

christian attacked muslims temple mount waving israeli flag via pamela geller  :real Tweet 

logitechg fire nation attacked :real Tweet 

christian attacked muslims temple mount waving israeli flag via pamela geller  :real Tweet 

metro nashville police simply best thank protecting us call via aol :fake Tweet 

peaceful happy twitter experience avalanche idiocy  :fake Tweet 

car receiving tube installation avalanche sound output hhw :real Tweet 

fucking true decapitated throw head avalanche  :fake Tweet 

one glance avalanche drops one look heartbeat stops oh walk moon get couldnt said better :fake Tweet 

photo repos

yeah hate cats tried pet cat hand bite marks scratch marks bleeding dogsarebetterthancats :fake Tweet 

aichmomanic wraps cloth wound still bleeding ill ok falls ground im going die :fake Tweet 

ears bleeding  :fake Tweet 

dudeandpal bad eyes ears bleeding :fake Tweet 

wouldnt want quit even bleeding :fake Tweet 

idkimnotfunny quickly blew mom doesnt get popular :fake Tweet 

lmfao snap blew :fake Tweet 

blew get follow back  :fake Tweet 

atlwtmgc damn blew :fake Tweet 

wow social media really blew shania twain concert :fake Tweet 

cieistea idk blew :fake Tweet 

 im always one phone call away many people done blew chances :fake Tweet 

well tweet blew fast :fake Tweet 

well blew sleeping :fake Tweet 

funnychubbyguy poor white guy mom made pizza wanted mac n cheese blew school sad story :fake Tweet 

wow bro blew quick believe hes funny clever person times like  :fake Tweet 

tokyodotcom didnt either till blew mentions lol :fake Tweet 

fucking around google maps work pulled 

photo bonesymcbones beartooth body bag x  :fake Tweet 

twotone crossbody smartphone bag shipped via dians daily deals twotone  :fake Tweet 

genuine leather man bag messenger fit ipad mini tablet case cross body air jp full rea û  :fake Tweet 

meekmill think time consulted kendricklamar cause drake done body bagging :fake Tweet 

people bagging rouseys body shit id love body like ridiculing probably dudes skinny jeans :fake Tweet 

bagging last spot body attack getin traintuesday lesmills :fake Tweet 

 dear lord forgive body bagging meekmill twice amen drake  :fake Tweet 

 otooledalton first cant hit stick shit second im body bagging sight  :fake Tweet 

drake really body bagging meek atm :fake Tweet 

remember halo interview abc wasnt time pamanian devil body bagging fools  :fake Tweet 

jerseys artist wolfgangjoc pays homage late great biggie smalls body bagging hypnotize beat  :fake Tweet 

drake kill meek mill started bagging dead body  :fake Tweet 

man start body bagging loca

...

new favorite hardwell amp dannic survivors remake flpfamily soundcloud :fake Tweet 

rt syedihusain penelopephoto man survived hiroshima entered living hell earth  :real Tweet 

text worth life hear stories survivors pledge text drive  :fake Tweet 

thenerdsofcolor georgetakei narrates new doc hiroshima wwii bombing survivors tales check  :real Tweet 

legendary dutch producer dannic making way denver bring progessive house sound  :fake Tweet 

shaunking check tribute survivors sexual assault law enforcement officers facebook :fake Tweet 

hiroshima schoolgirls archive memories abomb horror via abcnews :real Tweet 

donation make difference lives survivors braininjury please donate today  :fake Tweet 

survivors isabel hodgins danny miller laura norton amp mark charnock take group pic summerfate behindthescenes  :fake Tweet 

crossborder terrorism pakistan caught redhanded viatimesofindia :real Tweet 

bbcnewsasia state sponsored terrorism :real Tweet 

judges considering fate children

meekmill beef idgaf went far used tragedy people died insult drake owe apology :fake Tweet 

love tragedy remedyyyy :fake Tweet 

jiwa mines rly tragedy mp live recount horror ûïwhen saw coaches train plunging water  :real Tweet 

rly tragedy mp live recount horror ûïwhen saw coaches train plunging water called  :real Tweet 

women bonded tragedy em broken way :real Tweet 

rly tragedy mp live recount horror ûïwhen saw coaches train plunging water called  :real Tweet 

rly tragedy mp live recount horror ûïwhen saw coaches train plunging water c :real Tweet 

rly tragedy mp live recount horror ûïwhen saw coaches train plunging water called  :real Tweet 

 sheilagunnreid war well never win r yezidi women trapped slaveswe know nothing evil :fake Tweet 

think reality tv show tell quell panic rising throat introvert trapped tokyo  :fake Tweet 

signed work week trapped nerve move life struggle right  :fake Tweet 

confused westie believes dogs trapped computer  :fake Tweet 

teambrianmundi

shortest hardest tongue twister toy boat try saying five times fast :fake Tweet 

struggling return life makurazaki typhoon september killed thousands  :real Tweet 

obama declares disaster typhoondevastated saipan obama signs disaster declaration northern marians  :real Tweet 

map typhoon soudelors predicted path approaches taiwan expected make landfall southern china û  :real Tweet 

obama signs disaster declaration northern marians typhoon hits us territory  :real Tweet 

typhoon soudelor heads taiwan wtf news tv fun funny :real Tweet 

uk weather alert remnants asian super typhoon soudelor cause violent  :real Tweet 

trending viral spectacular satellite view typhoon soudelor rumblevideo  :real Tweet 

worldnews world saipan water electricity typhoon aftermath voice america world google news  :real Tweet 

striking views super typhoon soudelor tracks toward taiwan china  :real Tweet 

saipan water electricity aftermath typhoon  :real Tweet 

obama declares disaster typhoondevastat

 breaking massive wild field fires reported throughout germany fires eastern german  :real Tweet 

complain pa atleast dont wild fires taking homes like north cali floods florida right :real Tweet 

wife opted take holidays amp historically since met lanahillman gets special treatment bonfires cant start wild firesrain :fake Tweet 

like blazing wild fires singing name :fake Tweet 

netnewsledger wild fire update ûò august thunder bay ûò weather ûò thereåêwere new fires confirmed  :real Tweet 

 becuase cause property damage cause wild fires :real Tweet 

hey macys moms house burned ca wild fires amp bought e gift card buy clothes cont  :fake Tweet 

california wild firesclearly caused global climate change excess  :real Tweet 

california wild fires blow mind every time :real Tweet 

 wild fires getting anywhere near :real Tweet 

stampedes fires brought home love make stay tame wild heart audiobook bynr beingauthor :fake Tweet 

go home california youre drunk natural selection taken 

 thought said saturday night near died lolol youll wrecked :fake Tweet 

wanna ease mind make feel alright :fake Tweet 

yakuboobs think deactivated notifications aids tesco wrecked lol :fake Tweet 

rt cnbc words disney ceo bob iger wrecked disneys stock says jim cramer  :fake Tweet 

smackdown tyme put good mood since got wrecked smh :fake Tweet 

thrillhho jsyk havent stopped thinking abt remus slumped bathroom door day wrecked  :fake Tweet 

stighefootball begovic garbage got wrecked red bull reserve team everyone else preseason :fake Tweet 

wrecked today got hattrick  :fake Tweet 

ebola ebolaoutbreak ebola virus birmingham ala firefighters quarantined possible exposure officials say  :real Tweet 

malaysian pm confirms debris missing flight  :real Tweet 

officials alabama home quarantined possible ebola case washington times :real Tweet 

see old pkk suicide bomber detonated bomb turkey army trench released harun ìàekdar msoreo :real Tweet 

conference attendees blue line airpo